In [1]:
println("Start")

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-D4GN0MQF:4040
SparkContext available as 'sc' (version = 2.3.0, master = local[*], app id = local-1564662866818)
SparkSession available as 'spark'


Start


In [5]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import spark.implicits._

import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import spark.implicits._


In [1]:
val left = Seq((0, "zero"), (1, "one")).toDF("id", "left")
val right = Seq((0, "zero"), (2, "two"), (3, "three")).toDF("id", "right")

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-D4GN0MQF:4040
SparkContext available as 'sc' (version = 2.3.0, master = local[*], app id = local-1566400781689)
SparkSession available as 'spark'


left: org.apache.spark.sql.DataFrame = [id: int, left: string]
right: org.apache.spark.sql.DataFrame = [id: int, right: string]


In [2]:
left.show

+---+----+
| id|left|
+---+----+
|  0|zero|
|  1| one|
+---+----+



In [3]:
right.show

+---+-----+
| id|right|
+---+-----+
|  0| zero|
|  2|  two|
|  3|three|
+---+-----+



In [4]:
left.join(right, "id").show

+---+----+-----+
| id|left|right|
+---+----+-----+
|  0|zero| zero|
+---+----+-----+



# 1 - Join and rank

In [4]:
val dfTries = Seq(
  ("Try 1", "2018-08-01"),
  ("Try 2", "2018-09-01"),
  ("Try 3", "2018-10-01"),
  ("Try 4", "2018-10-02")
).toDF("try", "try_day")

dfTries: org.apache.spark.sql.DataFrame = [try: string, try_day: string]


In [7]:
val dfContracts = Seq(
  ("contract 1", "2018-08-01"),
  ("contract 2", "2018-09-02"),
  ("contract 3", "2018-10-01")
).toDF("contract", "contract_day")

dfContracts: org.apache.spark.sql.DataFrame = [contract: string, contract_day: string]


In [13]:
dfTries.
    withColumn("try_ts", unix_timestamp($"try_day", "yyyy-MM-dd")).
    withColumn("prev_try_ts", lead($"try_ts",1).over(Window.orderBy($"try"))).
    withColumn("rank", when(
        $"prev_try_ts".isNull || abs($"try_ts" - $"prev_try_ts") > 10 * 24 * 3600,
        1
    ).otherwise(2)
).
join(
   dfContracts,
   $"rank" === 1 && abs($"try_ts" - unix_timestamp($"contract_day", "yyyy-MM-dd")) <= 2 * 24 * 3600,
        "left_outer").show

2019-08-01 09:53:25 WARN  WindowExec:66 - No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+-----+----------+----------+-----------+----+----------+------------+
|  try|   try_day|    try_ts|prev_try_ts|rank|  contract|contract_day|
+-----+----------+----------+-----------+----+----------+------------+
|Try 1|2018-08-01|1533092400| 1535770800|   1|contract 1|  2018-08-01|
|Try 2|2018-09-01|1535770800| 1538362800|   1|contract 2|  2018-09-02|
|Try 3|2018-10-01|1538362800| 1538449200|   2|      null|        null|
|Try 4|2018-10-02|1538449200|       null|   1|contract 3|  2018-10-01|
+-----+----------+----------+-----------+----+----------+------------+

